# R packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import iqr, norm
import statsmodels.stats.api as sms
import arch 
import pmdarima.arima as pm
from statsmodels.tsa.arima_model import ARIMA

import statsmodels.tsa.seasonal as tsa
import statsmodels.graphics.tsaplots as tsap 
from statsmodels.tsa.stattools import adfuller

# Recursive OLS residuals
from statsmodels.stats.diagnostic import recursive_olsresiduals
from statsmodels.stats.diagnostic import breaks_cusumolsresid
import chow_test
from scipy import stats
import arch


from fredapi import Fred
fred = Fred(api_key='ca6ea25f72db47e2d070a76b7e9e639b')

import warnings
warnings.filterwarnings("ignore")
np.random.seed(432)

In [5]:
# #library to use R in Python 
# # To start the R session in the notebook
# import rpy2

# import warnings
# warnings.filterwarnings('ignore')

# from rpy2.robjects import pandas2ri
# import rpy2.rinterface as rinterface
# pandas2ri.activate()

In [2]:
import rpy2
print(rpy2.__version__)

3.5.3


In [3]:
import rpy2.situation
for row in rpy2.situation.iter_info():
    print(row)

Unable to determine R home: [WinError 2] The system cannot find the file specified


rpy2 version:
3.5.3
Python version:
3.7.11 (default, Jul 27 2021, 09:42:29) [MSC v.1916 64 bit (AMD64)]
Looking for R's HOME:
    Environment variable R_HOME: None
    InstallPath in the registry: C:\Program Files\R\R-4.0.2
    Environment variable R_USER: None
    Environment variable R_LIBS_USER: None
R version:
    In the PATH: R version 4.0.2 (2020-06-22) -- "Taking Off Again"
    Loading R library from rpy2: OK
Additional directories to load R packages from:
None
C extension compilation:


In [4]:
# Importing the top-level sub-package is also initializing and starting R embedded in the current Python process:
import rpy2.robjects as robjects

In [7]:
from rpy2.robjects.packages import importr
# import R's "base" package
base = importr('base')

# import R's "utils" package
utils = importr('utils')

Downloading and installing R packages is usually performed by fetching R packages from a package repository and installing them locally. Capabilities to do this are provided by R libraries, and when in Python we can simply use them using rpy2. An interface to the R features is provided in rpy2.robjects.packages (where the function importr() introduced above is defined).

In [10]:
# import rpy2's package module
import rpy2.robjects.packages as rpackages

# import R's utility package
utils = rpackages.importr('utils')

# select a mirror for R packages
utils.chooseCRANmirror(ind=1) # select the first mirror in the list

<rpy2.rinterface_lib.sexp.NULLType object at 0x0000016821274B08> [RTYPES.NILSXP]

In [12]:
# utils.install_packages('rugarch')

In [13]:
# R package names
packnames = ('ggplot2', 'hexbin', 'rmgarch')

# R vector of strings
from rpy2.robjects.vectors import StrVector

# Selectively install what needs to be install.
# We are fancy, just because we can.
names_to_install = [x for x in packnames if not rpackages.isinstalled(x)]
if len(names_to_install) > 0:
    utils.install_packages(StrVector(names_to_install))

R[write to console]: Installing package into 'C:/Users/USUARIO/Documents/R/win-library/4.0'
(as 'lib' is unspecified)

R[write to console]: also installing the dependencies 'gmp', 'Rmpfr', 'bit', 'Bessel', 'ff', 'shape', 'pcaPP', 'corpcor'





  There are binary versions available but the source versions are later:
      binary source needs_compilation
Rmpfr  0.8-7  0.8-9              TRUE
ff     4.0.5  4.0.7              TRUE
pcaPP 1.9-74  2.0-2              TRUE

  Binaries will be installed


R[write to console]: trying URL 'https://cloud.r-project.org/bin/windows/contrib/4.0/gmp_0.6-5.zip'

R[write to console]: Content type 'application/zip'
R[write to console]:  length 1221918 bytes (1.2 MB)

R[write to console]: downloaded 1.2 MB


R[write to console]: trying URL 'https://cloud.r-project.org/bin/windows/contrib/4.0/Rmpfr_0.8-7.zip'

R[write to console]: Content type 'application/zip'
R[write to console]:  length 1659757 bytes (1.6 MB)

R[write to console]: downloaded 1.6 MB


R[write to console]: trying URL 'https://cloud.r-project.org/bin/windows/contrib/4.0/bit_4.0.4.zip'

R[write to console]: Content type 'application/zip'
R[write to console]:  length 630708 bytes (615 KB)

R[write to console]: downloaded 615 KB


R[write to console]: trying URL 'https://cloud.r-project.org/bin/windows/contrib/4.0/Bessel_0.6-0.zip'

R[write to console]: Content type 'application/zip'
R[write to console]:  length 304227 bytes (297 KB)

R[write to console]: downloaded 297 KB


R[write t

package 'gmp' successfully unpacked and MD5 sums checked
package 'Rmpfr' successfully unpacked and MD5 sums checked
package 'bit' successfully unpacked and MD5 sums checked
package 'Bessel' successfully unpacked and MD5 sums checked
package 'ff' successfully unpacked and MD5 sums checked
package 'shape' successfully unpacked and MD5 sums checked
package 'pcaPP' successfully unpacked and MD5 sums checked
package 'corpcor' successfully unpacked and MD5 sums checked
package 'rmgarch' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\USUARIO\AppData\Local\Temp\RtmpqqxaaF\downloaded_packages


In [16]:
robjects.r('''
        # create a function `f`
        f <- function(r, verbose=FALSE) {
            if (verbose) {
                cat("I am calling f().\n")
            }
            2 * pi * r
        }
        # call the function `f` with argument value 3
        f(3)
        ''')

18.849556


In [18]:
r_f = robjects.globalenv['f']
print(r_f.r_repr())

function (r, verbose = FALSE) 
{
    if (verbose) {
        cat("I am calling f().\n")
    }
    2 * pi * r
}


In [20]:
r_f(4)

25.132741


In [31]:
# Creating rpy2 vectors
res = robjects.StrVector(['abc', 'def'])
print(res)
print(res.r_repr())

print("")
res = robjects.IntVector([1, 2, 3])
print(res[:])
print(res.r_repr())

print("")
res = robjects.FloatVector([1.1, 2.2, 3.3])
print(res)
print(res.r_repr())


[1] "abc" "def"

c("abc", "def")

[1] 1 2 3

1:3

[1] 1.1 2.2 3.3

c(1.1, 2.2, 3.3)


In [33]:
# R matrixes and arrays are just vectors with a dim attribute.
v = robjects.FloatVector([1.1, 2.2, 3.3, 4.4, 5.5, 6.6])
m = robjects.r['matrix'](v, ncol = 2) #nrow = 3
print(m)

     [,1] [,2]
[1,]  1.1  4.4
[2,]  2.2  5.5
[3,]  3.3  6.6



In [34]:
# CALLING R FUNCTIONS
rsum = robjects.r['sum']
rsum(robjects.IntVector([1,2,3]))[0]

6

In [37]:
rsort = robjects.r['sort']
res = rsort(robjects.IntVector([1,2,3]), decreasing=True)
print(res)#.r_repr()

[1] 3 2 1



In [39]:
# Getting help
from rpy2.robjects.packages import importr
utils = importr("utils")
help_doc = utils.help("help")
help_doc[0]
str(help_doc)

R[write to console]: starting httpd help server ...
R[write to console]:  done



''

In [43]:
# help_where = utils.help_search("help")
# help_where

Examples

In [4]:
import rpy2.robjects as robjects

r = robjects.r

x = robjects.IntVector(range(10))
y = r.rnorm(10)

r.X11()

r.layout(r.matrix(robjects.IntVector([1,2,3,2]), nrow=2, ncol=2))
r.plot(r.runif(10), y, xlab="runif", ylab="foo/bar", col="red")

<rpy2.rinterface_lib.sexp.NULLType object at 0x0000019BCAE41F48> [RTYPES.NILSXP]

In [5]:
import rpy2.robjects as robjects
from rpy2.robjects import Formula, Environment
from rpy2.robjects.vectors import IntVector, FloatVector
from rpy2.robjects.lib import grid
from rpy2.robjects.packages import importr, data
from rpy2.rinterface_lib.embedded import RRuntimeError
import warnings

# The R 'print' functionb
rprint = robjects.globalenv.find("print")
stats = importr('stats')
grdevices = importr('grDevices')
base = importr('base')
datasets = importr('datasets')

In [6]:
import math, datetime
import rpy2.robjects.lib.ggplot2 as ggplot2
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
base = importr('base')

mtcars = data(datasets).fetch('mtcars')['mtcars']

In [7]:
pp = (ggplot2.ggplot(mtcars) +
      ggplot2.aes_string(x='wt', y='mpg', col='factor(cyl)') +
      ggplot2.geom_point() +
      ggplot2.geom_smooth(ggplot2.aes_string(group = 'cyl'),
                          method = 'lm'))
pp.plot()

R[write to console]: `geom_smooth()` using formula 'y ~ x'



: 